In [38]:
import backtrader as bt
import backtrader.indicators as btind
import backtrader.feeds as btfeeds

import pandas as pd
import pandas_datareader.data as web
import datetime

In [22]:
aapl = web.DataReader(
    name = 'AAPL',
    data_source = 'yahoo',
    start = '2017-01-01',
    end = '2017-12-31'  
).rename(
    columns = {
        'Open': 'open',
        'High': 'high',
        'Low': 'low',
        'Close': 'close',
        'Volume': 'volume'
    }
    
)
aapl.head()

,open,high,low,close,Adj Close,volume
Date,,,,,,
2017-01-03,115.800003,116.330002,114.760002,116.150002,110.691154,28781900
2017-01-04,115.849998,116.510002,115.750000,116.019997,110.567276,21118100
2017-01-05,115.919998,116.860001,115.809998,116.610001,111.129562,22193600
2017-01-06,116.779999,118.160004,116.470001,117.910004,112.368469,31751900
2017-01-09,117.949997,119.430000,117.940002,118.989998,113.397697,33561900


In [41]:
class MyStrategy(bt.Strategy):
    
    params = dict(
        period1 = 20,
        period2 = 25,
        period3 = 10,
        period4 = 50
    )
    
    def __init__(self):
        
        sma1 = btind.SimpleMovingAverage(
            self.datas[0], period = self.p.period1
        )
        
        sma2 = btind.SimpleMovingAverage(
            sma1, period = self.p.period2
        )
        
        something = btind.SimpleMovingAverage(
            sma2 - sma1 + self.data.close
        )
        
        sma3 = btind.SimpleMovingAverage(
            something, period = self.p.period3
        )
        
        greater = sma3 > sma1
        
        sma4 = btind.SimpleMovingAverage(
            greater, period = self.p.period4
        )

        df = pd.DataFrame([sma1, sma2])
        print(df)
        
        
cerebro = bt.Cerebro()

cerebro.addstrategy(MyStrategy)


data = btfeeds.PandasData(
    dataname = aapl,
    fromdate = datetime.datetime(2017, 1, 1),
    todate = datetime.datetime(2017, 12, 31)
)

cerebro.adddata(data)

print('Starting portfolio value: {}'.format(cerebro.broker.get_value()))
cerebro.run()
print('Final portfolio value: {}'.format(cerebro.broker.getvalue()))

Starting portfolio value: 10000.0
                                                   0
0  <backtrader.indicators.sma.SimpleMovingAverage...
1  <backtrader.indicators.sma.SimpleMovingAverage...
Final portfolio value: 10000.0
